This Notebook compares recommendations made by ChatGPT based on several prompts.

In [11]:
import pandas as pd

Building the Training Data

In [12]:
train = pd.read_csv('../data/train.csv')

In [13]:
train.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0


The movies each user has watched in the training data

In [14]:
user_movies = train.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

In [15]:
""" Number of users to run the recommender on
    (Use a small sample for faster results)
    If you want to run the recommender on all users, uncomment the last line
"""
num_users = 20

# Uncomment below to run on all users
# n = user_movies.shape[0]

The most recent movies each user has watched.

In [16]:
def get_user_recent_movies(dataframe, n = 10):
    return dataframe.groupby('user_id').apply(lambda x: x['movie_title'].tail(n).to_list())

user_recent_movies = get_user_recent_movies(train)

The movies each user rated highly.

In [17]:
def get_user_top_movies(dataframe, n = 10):
    return dataframe.groupby('user_id').apply(lambda x: x.sort_values('rating', ascending=False).head(n)['movie_title'].to_list())

user_top_movies = get_user_top_movies(train)

The genres of each movie.

In [18]:
movie_genres = train.groupby('movie_title').apply(lambda x: x['genres'].unique().tolist()).apply(lambda x: x[0].split(", "))

In [19]:
user_top_movies.head()

user_id
1     [Empire Strikes Back, The (1980), Usual Suspec...
5     [Wrong Trousers, The (1993), This Is Spinal Ta...
6     [Down by Law (1986), Graduate, The (1967), Lon...
8     [Contact (1997), Die Hard (1988), Braveheart (...
10    [Amadeus (1984), Taxi Driver (1976), All About...
dtype: object

The genres for each users top rated movies

In [20]:
user_top_movie_genres = user_top_movies.apply(lambda x: pd.Series(movie_genres.loc[x].sum()).unique().tolist())

The ratings provided by each user to every movie they watched in the training data.

In [21]:
user_ratings = train.drop_duplicates(['user_id', 'movie_title']).pivot(index='user_id', columns='movie_title', values='avg_rating').fillna(0)

In [22]:
from scipy.sparse import csr_matrix
sparse_user_ratings = csr_matrix(user_ratings)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Identifying similar users based on cosine similarity

In [24]:
def get_similar_users(sparse_user_ratings, index, n = 10):
    return pd.DataFrame(cosine_similarity(sparse_user_ratings) - np.identity(sparse_user_ratings.shape[0]), index = index, columns = index).apply(lambda x: list(x.sort_values(ascending = False).head(n).index), axis=1)

similar_users = get_similar_users(sparse_user_ratings, user_ratings.index)

In [25]:
similar_users.head()

user_id
1     [916, 457, 268, 435, 429, 823, 301, 276, 889, ...
5      [648, 407, 307, 497, 268, 276, 22, 622, 804, 70]
6       [18, 194, 716, 10, 666, 151, 321, 85, 389, 854]
8      [746, 158, 37, 352, 638, 425, 22, 627, 671, 538]
10      [6, 406, 666, 389, 321, 398, 524, 716, 194, 18]
dtype: object

Identifying Candidate movies which are the movies each user's similar users rated most highly overall.

In [26]:
def get_candidate_movies(similar_users, dataframe, n = 20):
    return similar_users.apply(lambda x: list(dataframe.loc[dataframe['user_id'].isin(x)].groupby('movie_title').sum().sort_values('rating', ascending = False).index[:n]))

candidate_movies = get_candidate_movies(similar_users, train)

In [27]:
user_prompt_data = pd.concat((user_movies, user_top_movies, user_recent_movies, similar_users, candidate_movies, user_top_movie_genres), axis = 1, keys = ['user_movies', 'user_top_movies', 'user_recent_movies', 'similar_users', 'candidate_movies', 'user_top_movie_genres'])

Collaborative Filtering Prompt

In [28]:
new_line = '\n'

def collabPrompt(each, user_prompt_data, new_line = '\n'):
    return f"""I am user {each}.
The most recent ten movies I have seen are:
{", ".join(user_prompt_data.loc[each, 'user_recent_movies'])}.
My top rated movies are:
{", ".join(user_prompt_data.loc[each, 'user_top_movies'])}.
The users who are most like me are {", ".join([str(each) for each in user_prompt_data.loc[each, 'similar_users']])}.
The top movies for each of these users are:
{new_line.join([f"{each}: {', '.join(user_prompt_data.loc[each, 'user_top_movies'])}" for each in user_prompt_data.loc[each, 'similar_users']])}.
Please recommend ten movies for me to watch that I have not seen. Provide brackets around your recommendations so I can easily parse them.
For example ([Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])"""
    

In [29]:
collab_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: collabPrompt(x, user_prompt_data))

In [30]:
from openai import OpenAI
from key import OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

In [31]:
collab_response = collab_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [70]:
collab_recommendations = collab_response.apply(lambda x: x.choices[0].message.content.split("[")[1].split("]")[0].split("\n"))

In [71]:
collab_recommendations.head()

user_id
1     [Midnight Cowboy (1969), Lost in Translation (...
5     [The Shawshank Redemption (1994), Pulp Fiction...
6                              [Midnight Cowboy (1969)]
8     [The Matrix (1999), Inception (2010), Fight Cl...
10    [Pulp Fiction (1994), Eternal Sunshine of the ...
Name: user_id, dtype: object

In [34]:
test = pd.read_csv('../data/test.csv')

In [35]:
user_movies_test = test.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

In [77]:
collab_hits = pd.concat((user_movies, user_movies_test, collab_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'collab_recommendations']).dropna(subset = 'collab_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Prompt Similar to Zero-Shot Paper (reduced to one prompt, added genres)

In [37]:
def genrePrompts(each, user_prompt_data, new_line = '\n'):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?.
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [38]:
genre_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: genrePrompts(x, user_prompt_data))

In [39]:
genre_prompts.head()

user_id
1     \nCandidate Set (candidate movies): ['Chasing ...
5     \nCandidate Set (candidate movies): ['Raiders ...
6     \nCandidate Set (candidate movies): ['Casablan...
8     \nCandidate Set (candidate movies): ['Blade Ru...
10    \nCandidate Set (candidate movies): ['Casablan...
Name: user_id, dtype: object

In [40]:
genre_response = genre_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [41]:
genre_recommendations = genre_response.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [42]:
genre_recommendations.head()

user_id
1     [Chasing Amy (1997), Raiders of the Lost Ark (...
5     [Monty Python and the Holy Grail (1974), Princ...
6     [Raiders of the Lost Ark (1981), Shawshank Red...
8     [Blade Runner (1982), Empire Strikes Back, The...
10    [Casablanca (1942), Wizard of Oz, The (1939), ...
Name: user_id, dtype: object

In [43]:
genre_hits = pd.concat((user_movies, user_movies_test, genre_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'genre_recommendations']).dropna(subset = 'genre_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Prompt Similar to Zero-Shot Paper (uses two prompts, adds genres)

In [44]:
def twoStepPrompt1(each, user_prompt_data):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: 
"""

def twoStepPrompt2(each, user_prompt_data, response1):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: {response1.loc[each]}.
Step 2: Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [45]:
prompt1 = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: twoStepPrompt1(x, user_prompt_data))

In [46]:
response1 = prompt1.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [47]:
response1.head()

user_id
1     ChatCompletion(id='chatcmpl-8UQ70ZT6KadcVO8AyU...
5     ChatCompletion(id='chatcmpl-8UQ72nx0X6CH6vixWA...
6     ChatCompletion(id='chatcmpl-8UQ74BxMNsNs2Mc4mC...
8     ChatCompletion(id='chatcmpl-8UQ75QgczHwggUnDbn...
10    ChatCompletion(id='chatcmpl-8UQ77Hkrss5frwXO25...
Name: user_id, dtype: object

In [48]:
prompt2 = pd.Series(user_prompt_data.index, index = user_prompt_data.index).iloc[:num_users].apply(lambda x: twoStepPrompt2(x, user_prompt_data, response1))

In [49]:
response2 = prompt2.apply(lambda x: client.chat.completions.create(
    model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [50]:
twoStep_recommendations = response2.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [51]:
twoStep_recommendations.head()

user_id
1     [Chasing Amy (1997), Raiders of the Lost Ark (...
5     [Monty Python and the Holy Grail (1974), Princ...
6     [Wizard of Oz, The (1939), Pulp Fiction (1994)...
8     [Blade Runner (1982), Empire Strikes Back, The...
10    [Casablanca (1942), Wizard of Oz, The (1939), ...
Name: user_id, dtype: object

In [52]:
twoStep_hits = pd.concat((user_movies, user_movies_test, twoStep_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'twoStep_recommendations']).dropna(subset = 'twoStep_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

In [53]:
twoStep_hits.head()

user_id
1     10
5      8
6     10
8      9
10    10
dtype: int64

Prompt using wikipedia movie summaries

In [54]:
movie_wiki = pd.read_csv('../data/movie_wiki.csv')
movie_wiki.head()

,movie_title,wiki_summary
0,'Til There Was You (1997),'Til There Was You is a 1997 American romantic...
1,1-900 (1994),1-900 or 06 is a 1994 Dutch erotic romantic dr...
2,101 Dalmatians (1996),101 Dalmatians is a 1996 American adventure co...
3,12 Angry Men (1957),12 Angry Men is a 1957 American legal drama fi...
4,187 (1997),One Eight Seven (also known as 187) is a 1997 ...


In [55]:
def wikiPrompt(each, user_prompt_data, movie_wiki, new_line = '\n'):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Summary of the movies I have watched: {new_line.join([f"{eachMovie}: {movie_wiki.loc[movie_wiki['movie_title'] == eachMovie, 'wiki_summary'].iloc[0]}" for eachMovie in user_prompt_data.loc[1, 'user_top_movies']])}
Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?.
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [56]:
wiki_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: wikiPrompt(x, user_prompt_data, movie_wiki))

In [57]:
wiki_prompts.head()

user_id
1     \nCandidate Set (candidate movies): ['Chasing ...
5     \nCandidate Set (candidate movies): ['Raiders ...
6     \nCandidate Set (candidate movies): ['Casablan...
8     \nCandidate Set (candidate movies): ['Blade Ru...
10    \nCandidate Set (candidate movies): ['Casablan...
Name: user_id, dtype: object

In [58]:
wiki_response = wiki_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [59]:
wiki_recommendations = wiki_response.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [60]:
wiki_recommendations.head()

user_id
1     [Chasing Amy (1997), Raiders of the Lost Ark (...
5     [Monty Python and the Holy Grail (1974), Princ...
6     [It Happened One Night (1934), Lawrence of Ara...
8     [Blade Runner (1982), Empire Strikes Back, The...
10    [Casablanca (1942), Wizard of Oz, The (1939), ...
Name: user_id, dtype: object

In [61]:
wiki_hits = pd.concat((user_movies, user_movies_test, wiki_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'wiki_recommendations']).dropna(subset = 'wiki_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Baseline Recommender: Recommend top 10 most popular movies to every user

In [62]:
viewings = train.groupby('movie_title').count().sort_values('user_id', ascending=False)

In [63]:
viewings.head()

,user_id,movie_id,rating,timestamp,genres,avg_rating
movie_title,,,,,,
Star Wars (1977),393,393,393,393,393,393
Fargo (1996),355,355,355,355,355,355
Return of the Jedi (1983),350,350,350,350,350,350
Liar Liar (1997),317,317,317,317,317,317
Toy Story (1995),315,315,315,315,315,315


In [64]:
top_10_movies = viewings.head(10).index.tolist()
top_10_movies

['Star Wars (1977)',
 'Fargo (1996)',
 'Return of the Jedi (1983)',
 'Liar Liar (1997)',
 'Toy Story (1995)',
 'English Patient, The (1996)',
 'Independence Day (ID4) (1996)',
 'Contact (1997)',
 'Scream (1996)',
 'Raiders of the Lost Ark (1981)']

In [65]:
baseline_hits = pd.concat((user_movies, user_movies_test), axis=1).dropna().iloc[:num_users].apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(top_10_movies)), axis = 1)

Comparison of Prompts

In [79]:
comparison = pd.concat((collab_hits, genre_hits, twoStep_hits, wiki_hits, baseline_hits), axis = 1, keys = ['collab_hits', 'genre_hits', 'twoStep_hits', 'wiki_hits', 'baseline_hits'])

Below values are Hit Rate in % (i.e. 100 is 100%)

In [80]:
comparison.describe().iloc[1:] / 10 * 100

,collab_hits,genre_hits,twoStep_hits,wiki_hits,baseline_hits
mean,2.500000,82.000000,72.500000,76.000000,65.000000
std,5.501196,17.947291,31.601965,26.437613,20.900768
min,0.000000,30.000000,0.000000,0.000000,20.000000
25%,0.000000,70.000000,67.500000,60.000000,60.000000
50%,0.000000,90.000000,80.000000,85.000000,70.000000
75%,0.000000,92.500000,92.500000,92.500000,72.500000
max,20.000000,100.000000,100.000000,100.000000,100.000000
